In [ ]:
# ================================================================
# Google Drive 年度フォルダZip圧縮 ＆ 元フォルダ削除スクリプト
# (GitHub Actions & Recursive Search Version)
# ================================================================

# --- ライブラリのインポート ---
import shutil
import os
import re
from datetime import datetime

# ================================================================
# ▼▼▼ 設定はここから ▼▼▼
# このセルは、後述のワークフローファイル(.yml)によって
# 実行時に値が自動的に設定（上書き）されます。
# ================================================================
# ★設定1★ 探索を開始するフォルダパス (rcloneがDLしたローカルパス)
search_start_path = 'drive_data'

# ★設定2★ 何年前の年度をアーカイブするか
years_ago = 3
# ================================================================
# ▲▲▲ 設定はここまで ▲▲▲
# ================================================================

print("\n--- Notebook内の処理を開始します ---")

if not os.path.exists(search_start_path):
    print(f"❌ エラー: 探索開始パス '{search_start_path}' が見つかりません。")
else:
    today = datetime.now()
    current_fiscal_year = today.year if today.month >= 4 else today.year - 1
    fiscal_year_to_archive = current_fiscal_year - years_ago

    # ★★★ 変更点 ★★★
    # フォルダ名のパターンをより具体的に指定
    target_folder_pattern = re.compile(rf'^{fiscal_year_to_archive}年度 \({fiscal_year_to_archive}-04_{fiscal_year_to_archive + 1}-03\)')

    print(f'{years_ago}年前の年度（{fiscal_year_to_archive}年度）をアーカイブします。')
    print(f'探索開始パス: {search_start_path}')
    print(f'対象フォルダの検索パターン: {target_folder_pattern.pattern}')

    found_folder_path = None

    # os.walkを使って再帰的にフォルダを探索
    for dirpath, dirnames, filenames in os.walk(search_start_path):
        for dirname in dirnames:
            if target_folder_pattern.match(dirname):
                found_folder_path = os.path.join(dirpath, dirname)
                print(f' -> 対象フォルダ「{found_folder_path}」を発見。')
                break # 最初の１つが見つかったら内側のループを抜ける
        if found_folder_path:
            break # 最初の１つが見つかったら外側のループも抜ける

    if found_folder_path:
        # Zipファイルを作成するパスを、見つかったフォルダと同じ階層に設定
        output_zip_path = found_folder_path + '.zip'

        print(f'  -> 圧縮前にフォルダ内のファイル形式をチェックします...')
        google_extensions = ('.gdoc', '.gsheet', '.gslides', '.gform', '.gdraw', '.gmap', '.gsite')
        problematic_files = []
        for root, dirs, files in os.walk(found_folder_path):
            for file in files:
                if file.lower().endswith(google_extensions):
                    problematic_files.append(os.path.join(root, file))

        if problematic_files:
            print(f'\n❌ エラー: 圧縮できないGoogle形式のファイルが検出されたため、処理を中断しました。')
            for f_path in problematic_files:
                print(f'  - {f_path}')
            raise ValueError("Google Docs format files found.")
        else:
            print(f'  -> ✅ ファイル形式は正常です。圧縮処理を開始します...')
            try:
                shutil.make_archive(base_name=found_folder_path, format='zip', root_dir=found_folder_path)
                print(f'    -> ✅ 圧縮完了: {output_zip_path}')

                # ワークフローに伝えるため、search_start_pathからの相対パスをファイルに書き出す
                relative_zip_path = os.path.relpath(output_zip_path, search_start_path)
                relative_folder_path = os.path.relpath(found_folder_path, search_start_path)

                with open('zip_to_upload.txt', 'w') as f:
                    f.write(relative_zip_path)
                with open('folder_to_delete.txt', 'w') as f:
                    f.write(relative_folder_path)

                print(f"    -> ✅ アップロード対象Zip: '{relative_zip_path}'")
                print(f"    -> ✅ 削除対象フォルダ: '{relative_folder_path}'")

            except Exception as e:
                print(f'    -> ❌ エラー: 圧縮中に問題が発生しました: {e}')
                raise e
    else:
        print(f' -> 対象の年度フォルダは見つかりませんでした。')

    print('\n🎉 Notebook内の処理が完了しました。')




--- Notebook内の処理を開始します ---
❌ エラー: 探索開始パス 'drive_data' が見つかりません。
